In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import models,layers

In [2]:
IMAGE_SIZE = 256
EPOCHS=15

BATCH_SIZE = 32

In [3]:
df=tf.keras.preprocessing.image_dataset_from_directory(
"/kaggle/input/face-mask-lite-dataset",
shuffle=True,
image_size=(IMAGE_SIZE,IMAGE_SIZE),
batch_size=BATCH_SIZE
)

Found 20000 files belonging to 2 classes.


2021-10-23 16:55:32.345657: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-23 16:55:32.435459: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-23 16:55:32.436132: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-23 16:55:32.447684: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [4]:
cn=df.class_names
cn

['with_mask', 'without_mask']

In [5]:
def get_dataset_partitions_tf(ds, train_split=0.7, val_split=0.1, test_split=0.2):
    #assert (train_split + test_split + val_split) == 1
    
    ds_size = len(ds)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [6]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(df)

In [7]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255),
])

In [8]:
in_shape=(32,IMAGE_SIZE,IMAGE_SIZE,3)
model1=models.Sequential(
[
    resize_and_rescale,
    layers.Conv2D(32,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(2, activation='softmax'),
     
]
)
model1.build(input_shape=in_shape)

In [9]:
model1.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [10]:
history = model1.fit(
    train_ds,
    batch_size=32,
    validation_data=val_ds,
    verbose=1,
    epochs=5,
)

Epoch 1/5


2021-10-23 16:55:35.610017: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-10-23 16:55:45.761884: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 245 of 256
2021-10-23 16:55:46.213208: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:228] Shuffle buffer filled.
2021-10-23 16:55:48.125306: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


437/437 [==============================] - 1134s 3s/step - loss: 0.0440 - accuracy: 0.9847 - val_loss: 5.9301e-04 - val_accuracy: 0.9995
Epoch 2/5


2021-10-23 17:15:07.644667: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 249 of 256
2021-10-23 17:15:07.863612: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:228] Shuffle buffer filled.


437/437 [==============================] - 1107s 3s/step - loss: 7.0542e-07 - accuracy: 1.0000 - val_loss: 2.7432e-07 - val_accuracy: 1.0000
Epoch 3/5
437/437 [==============================] - 1130s 3s/step - loss: 3.5514e-08 - accuracy: 1.0000 - val_loss: 8.8202e-08 - val_accuracy: 1.0000
Epoch 4/5
437/437 [==============================] - ETA: 0s - loss: 1.0784e-08 - accuracy: 1.0000

2021-10-23 18:02:06.895016: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 253 of 256
2021-10-23 18:02:07.023961: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:228] Shuffle buffer filled.


437/437 [==============================] - 1204s 3s/step - loss: 1.0784e-08 - accuracy: 1.0000 - val_loss: 4.5844e-08 - val_accuracy: 1.0000
Epoch 5/5
437/437 [==============================] - 1130s 3s/step - loss: 5.4217e-09 - accuracy: 1.0000 - val_loss: 3.1785e-08 - val_accuracy: 1.0000


In [11]:
scores=model1.evaluate(test_ds)

2021-10-23 18:31:20.989243: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 255 of 256
2021-10-23 18:31:21.003954: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:228] Shuffle buffer filled.


126/126 [==============================] - 734s 1s/step - loss: 3.2242e-07 - accuracy: 1.0000


In [12]:
model1.save("maskmodel4.h5")
model1.save("maskmodel4")

2021-10-23 18:43:26.881199: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
